In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
import pyvo as vo
import pickle
from astroquery.vizier import Vizier
import concurrent.futures
import requests
import os
from tqdm import tqdm

In [10]:
import concurrent.futures
import requests

obsid_list = gal_lamost_data['obsid'].values

# Define the list of URLs to download
urls = [f"https://www.lamost.org/dr7/v2.0/spectrum/fits/{obsid}" for obsid in obsid_list]

def download_file(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
        file_name = url.split('/')[-1]
        with open(file_name, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded {file_name}")
    except requests.HTTPError as e:
        print(f"Failed to download {url}: {e}")

# Use ThreadPoolExecutor to download files in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(download_file, urls)

Failed to download https://www.lamost.org/dr7/v2.0/spectrum/fits/779516010: 500 Server Error:  for url: https://www.lamost.org/dr7/v2.0/spectrum/fits/779516010
Downloaded 161810188
Downloaded 475404020
Downloaded 215213
Downloaded 252906109
Downloaded 161802241
Downloaded 56212124
Downloaded 285410168
Downloaded 188002157
Failed to download https://www.lamost.org/dr7/v2.0/spectrum/fits/797010236: 500 Server Error:  for url: https://www.lamost.org/dr7/v2.0/spectrum/fits/797010236
Failed to download https://www.lamost.org/dr7/v2.0/spectrum/fits/797005026: 500 Server Error:  for url: https://www.lamost.org/dr7/v2.0/spectrum/fits/797005026
Failed to download https://www.lamost.org/dr7/v2.0/spectrum/fits/207512059: 500 Server Error:  for url: https://www.lamost.org/dr7/v2.0/spectrum/fits/207512059
Downloaded 82411233
Downloaded 677516079
Failed to download https://www.lamost.org/dr7/v2.0/spectrum/fits/866301202: 500 Server Error:  for url: https://www.lamost.org/dr7/v2.0/spectrum/fits/86630

In [5]:
# Load your GAL data and LAMOST catalog (assuming you have a local CSV or FITS file for LAMOST)
gal_data = pd.read_pickle("gal_data.pkl")  # Loaded GAL data
lamost_catalog = pd.read_csv("dr9_v2.0_LRS_catalogue.csv")  # Assuming CSV format for LAMOST catalog

# Ensure that RA and Dec columns are numeric and have units
gal_data['ra'] = pd.to_numeric(gal_data['ra'], errors='coerce')  # Coerce non-numeric to NaN
gal_data['dec'] = pd.to_numeric(gal_data['dec'], errors='coerce')
lamost_catalog['ra'] = pd.to_numeric(lamost_catalog['ra'], errors='coerce')
lamost_catalog['dec'] = pd.to_numeric(lamost_catalog['dec'], errors='coerce')

# Drop rows with NaN values in RA or Dec
gal_data = gal_data.dropna(subset=['ra', 'dec'])
lamost_catalog = lamost_catalog.dropna(subset=['ra', 'dec'])

# Convert GAL and LAMOST data to SkyCoord objects for crossmatching
gal_coords = SkyCoord(ra=gal_data['ra'].values*u.deg, dec=gal_data['dec'].values*u.deg)
lamost_coords = SkyCoord(ra=lamost_catalog['ra'].values*u.deg, dec=lamost_catalog['dec'].values*u.deg)

# Perform the crossmatch using astropy's match_to_catalog_sky function
idx, d2d, _ = gal_coords.match_to_catalog_sky(lamost_coords)

# Define a matching radius
match_radius = 1 * u.arcsec
matches = d2d < match_radius

# Filter the matches
matched_gal = gal_data.iloc[matches]
matched_lamost = lamost_catalog.iloc[idx[matches]]

# Combine matched data
gal_lamost_data = pd.concat([matched_gal.reset_index(drop=True), matched_lamost.reset_index(drop=True)], axis=1)

# Save the crossmatched data
gal_lamost_data.to_pickle("gal_lamost_data.pkl")

print(f"Number of matches: {gal_lamost_data.shape[0]}")
print("out of ", gal_data.shape[0])

Number of matches: 2533
out of  33531


In [6]:
# Load the crossmatched data
gal_lamost_data = pd.read_pickle("gal_lamost_data.pkl")

# Define the list of LAMOST observation IDs (obsid) to download
obsid_list = gal_lamost_data['obsid'].values

# Specify the folder to save the files
save_folder = "gal_spectra"

# Create the folder if it doesn't exist
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# List of URLs to download
urls = [f"https://www.lamost.org/dr7/v2.0/spectrum/fits/{obsid}" for obsid in obsid_list]

def download_file(url):
    try:
        file_name = os.path.join(save_folder, url.split('/')[-1])
        
        # Check if the file already exists to avoid re-downloading
        if not os.path.exists(file_name):
            response = requests.get(url)
            response.raise_for_status()  # Check for HTTP errors
            
            with open(file_name, 'wb') as f:
                f.write(response.content)
            return f"Downloaded {file_name}"
        else:
            return f"{file_name} already exists"
    except requests.HTTPError as e:
        return f"Failed to download {url}: {e}"

# Main download function using ThreadPoolExecutor with a progress bar
def download_all_files(urls):
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        # Create a progress bar with total number of URLs
        with tqdm(total=len(urls), desc="Downloading", unit="file") as pbar:
            futures = {executor.submit(download_file, url): url for url in urls}
            
            for future in concurrent.futures.as_completed(futures):
                result = future.result()
                pbar.update(1)  # Update the progress bar for each completed download
                print(result)

# Call the function to start downloading
download_all_files(urls)


Downloading:   0%|          | 1/2533 [00:01<51:06,  1.21s/file]

Failed to download https://www.lamost.org/dr7/v2.0/spectrum/fits/779516010: 500 Server Error:  for url: https://www.lamost.org/dr7/v2.0/spectrum/fits/779516010


Downloading:   0%|          | 2/2533 [00:01<37:05,  1.14file/s]

Failed to download https://www.lamost.org/dr7/v2.0/spectrum/fits/866301202: 500 Server Error:  for url: https://www.lamost.org/dr7/v2.0/spectrum/fits/866301202


Downloading:   0%|          | 3/2533 [00:02<24:29,  1.72file/s]

...

Downloading:  69%|██████▉   | 1749/2533 [08:46<26:47,  2.05s/file]

Downloaded gal_spectra\438809039


Downloading:  69%|██████▉   | 1750/2533 [08:49<26:51,  2.06s/file]

Downloaded gal_spectra\438804026


Downloading:  69%|██████▉   | 1750/2533 [08:49<03:57,  3.30file/s]


## STAR SPECTRA

In [2]:
# Load your STAR data and LAMOST catalog 
star_data = pd.read_pickle("star_data.pkl")  # Loaded STAR data
lamost_catalog = pd.read_csv("dr9_v2.0_LRS_catalogue.csv")  # Assuming CSV format

# Ensure that RA and Dec columns are numeric and have units
star_data['ra'] = pd.to_numeric(star_data['ra'], errors='coerce')  # Coerce non-numeric to NaN
star_data['dec'] = pd.to_numeric(star_data['dec'], errors='coerce')
lamost_catalog['ra'] = pd.to_numeric(lamost_catalog['ra'], errors='coerce')
lamost_catalog['dec'] = pd.to_numeric(lamost_catalog['dec'], errors='coerce')

# Drop rows with NaN values in RA or Dec
star_data = star_data.dropna(subset=['ra', 'dec'])
lamost_catalog = lamost_catalog.dropna(subset=['ra', 'dec'])

# Convert STAR and LAMOST data to SkyCoord objects for crossmatching
star_coords = SkyCoord(ra=star_data['ra'].values*u.deg, dec=star_data['dec'].values*u.deg)
lamost_coords = SkyCoord(ra=lamost_catalog['ra'].values*u.deg, dec=lamost_catalog['dec'].values*u.deg)

# Perform the crossmatch using astropy's match_to_catalog_sky function
idx, d2d, _ = star_coords.match_to_catalog_sky(lamost_coords)

# Define a matching radius
match_radius = 1 * u.arcsec
matches = d2d < match_radius

# Filter the matches    
matched_star = star_data.iloc[matches]
matched_lamost = lamost_catalog.iloc[idx[matches]]

# Combine matched data
star_lamost_data = pd.concat([matched_star.reset_index(drop=True), matched_lamost.reset_index(drop=True)], axis=1)

# Save the crossmatched data
star_lamost_data.to_pickle("star_lamost_data.pkl")

print(f"Number of matches: {star_lamost_data.shape[0]}")
print("out of ", star_data.shape[0])


Number of matches: 94651
out of  1499508


In [3]:
import concurrent.futures
import requests
import os
from tqdm import tqdm

# Load the crossmatched data
star_lamost_data = pd.read_pickle("star_lamost_data.pkl")
obsid_list = star_lamost_data['obsid'].values

# Specify the folder to save the files
save_folder = "star_spectra"

# Create the folder if it doesn't exist
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# List of URLs to download (assuming obsid_list is available)
urls = [f"https://www.lamost.org/dr7/v2.0/spectrum/fits/{obsid}" for obsid in obsid_list]

# Function to download a file from a URL
def download_file(url):
    try:
        file_name = os.path.join(save_folder, url.split('/')[-1])
        
        # Check if the file already exists to avoid re-downloading
        if not os.path.exists(file_name):
            response = requests.get(url)
            response.raise_for_status()  # Check for HTTP errors
            
            with open(file_name, 'wb') as f:
                f.write(response.content)
            return True, file_name  # Success
        else:
            return True, file_name  # File already exists, considered successful
    except requests.HTTPError as e:
        return False, f"Failed to download {url}: {e}"  # Failure

# Main function to download all files with counters and progress bar
def download_all_files(urls):
    success_count = 0
    failure_count = 0

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        # Create a progress bar with total number of URLs
        with tqdm(total=len(urls), desc="Downloading", unit="file") as pbar:
            futures = {executor.submit(download_file, url): url for url in urls}
            
            for future in concurrent.futures.as_completed(futures):
                success, result = future.result()
                pbar.update(1)  # Update the progress bar for each completed download
                
                if success:
                    success_count += 1
                    #print(f"Downloaded: {result}")
                else:
                    failure_count += 1
                    #print(f"Error: {result}")

    # Print final counts for successful and failed downloads
    print(f"\nDownload complete! Successfully downloaded: {success_count}, Failed: {failure_count}")

# Call the function to start downloading
download_all_files(urls)


Downloading:   2%|▏         | 2344/94651 [12:43<7:28:19,  3.43file/s] 

In [7]:
gal_lamost_data.shape



(2533, 56)